In [2]:
# import clickhouse_connect
# from qdrant_client import QdrantClient
# from langchain_qdrant import QdrantVectorStore
# from langchain_huggingface import HuggingFaceEmbeddings
# from sentence_transformers import SentenceTransformer
# from langchain_core.documents import Document
# from langchain_core.runnables import Runnable, RunnablePassthrough, chain, RunnableConfig
# import requests
# import logging
# import sys
# import os

# from pydantic import BaseModel, Field

# from typing import List
# import os
# import uuid

# from dotenv import load_dotenv
# load_dotenv()

# QDRANT_URL = os.getenv("QDRANT_URL")

# # api key in env variable QDRANT__SERVICE__API_KEY
# client_qd = QdrantClient(url=QDRANT_URL)

# CH_USER=os.getenv("CH_USER")
# CH_PASS=os.getenv("CH_PASS")
# CH_IP=os.getenv("CH_IP")
# CH_PORT=os.getenv("CH_PORT")  
# CH_DB=os.getenv("CH_DB") 

# # Initialize ClickHouse client
# client_ch = clickhouse_connect.get_client(
#     host=CH_IP,
#     port=int(os.getenv("CH_PORT")),     
#     username=CH_USER,
#     password=CH_PASS,
#     database=CH_DB
# )

# # Initialize LangSmith project
# os.environ["LANGSMITH_PROJECT"] = 'air_quality'

# logging.basicConfig(
#     level=logging.INFO,
#     format="%(asctime)s [%(levelname)s] %(message)s",
#     handlers=[
#         logging.FileHandler("app.log"),          # File output
#         logging.StreamHandler()                  # Console output
#     ],
#     force=True  # This overrides any prior logging config
# )

# # import tools
# # Get the parent directory
# parent_dir = os.path.abspath(os.path.join(os.getcwd(), ".."))

# # find tools in parent dir
# if os.path.isdir(os.path.join(parent_dir, 'tools')):
#     # Add parent directory to sys.path if found
#     sys.path.append(parent_dir)
    
# else:
#     # for run in spark
#     parent_dir = os.path.abspath(os.path.join(os.getcwd(), "../airflow/airflow_data"))
    
#     # Add parent directory to sys.path
#     sys.path.append(parent_dir)


# from tools import pd_tools
# from tools.paths import Paths
# from tools.db_tools import DbTools


